In [7]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font color="red" size="5">ch11. 데이터프레임과 시리즈(Pandas)</font></b>

# 9절. 데이터 그룹화 및 집계

In [8]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Goupby
cf. pivot_table

In [10]:
# 1) 그룹화 방법1
#      df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].기초통계함수()
#             [함수적용해서 출력할 열들] 생략시 그룹핑할열을 제외한 모든 열 적용  
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
15,5.7,4.4,1.5,0.4,setosa
29,4.7,3.2,1.6,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
56,6.3,3.3,4.7,1.6,versicolor
76,6.8,2.8,4.8,1.4,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
105,7.6,3.0,6.6,2.1,virginica
146,6.3,2.5,5.0,1.9,virginica


In [11]:
display(iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']])
display(iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean())

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [12]:
iris_grouped_sum = iris.groupby('Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [13]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [14]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [15]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') | 
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [16]:
#      df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법2
#     df.pivot_table(index=[그룹핑할열],
    #                values=[함수적용해서 출력할 열들],
    #                aggfunc=함수(기본값:mean))

In [17]:
iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

C:\Users\Admin\AppData\Local\Temp\ipykernel_17632\3702361755.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [18]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 다중열로 그룹화

In [19]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [20]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [21]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [22]:
# 각열의 임의의 데이터에 결측치를 할당
iris.iloc[0, 0] = np.nan

In [23]:
import random
print(random.sample(range(149), 3))
for col in range(iris.shape[1]-2): # iris.shape[1]-2 : 4
    iris.iloc[random.sample(range(150), col+1), col] = np.nan

[131, 81, 125]


In [24]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  148 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [25]:
iris.groupby(by=['Species','num'], 
            # as_index=False
            )[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              24           25
           1              24           25
versicolor 2              24           23
           3              25           25
virginica  4              25           23
           5              25           25

In [26]:
iris.pivot_table(index=['Species', 'num'],
                values = ['Petal.Length', 'Petal.Width'],
                aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              24           25
           1              24           25
versicolor 2              24           23
           3              25           25
virginica  4              25           23
           5              25           25

In [27]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  48.000000   50.000000  50.000000
             mean    5.004167    5.936000   6.588000
             std     0.359644    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  49.000000   50.000000  49.000000
             mean    3.426531    2.770000   2.973469
             std     0.382849    0.313798   0.325817
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.700000    3.000000   3.200000
             max     4.400000    3.400000   3.800000
Petal.Length count  48.000000   49.000000  50.000000
             mean    1.466667    4.253061   5.552000
             std     0.173000    0.472186   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.300000   5.550000
             75%     1.600000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
Petal.Width  count  50.000000   48.000000  48.000000
             mean    0.246000    1.325000   2.037500
             std     0.105386    0.201589   0.274152
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [28]:
iris.drop('num', axis=1, inplace=True)

In [29]:
# 레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,NaN,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- 판다스 공식 문서의 melt : https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [30]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                  package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [31]:
air_melted =  airquality.melt(id_vars=['Month','Day'],
               var_name='변수명', # 기본값:variable
               value_name='값') # 기본값 : value
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [32]:
# melt 연습하기 (부동산 와이드포맷 데이터를 롱포맷) \\192.168.0.25
%ls C:\ai_x\download\shareData\부동산_250213\

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 C:\ai_x\download\shareData\부동산_250213 디렉터리

2025-04-30  오후 05:24    <DIR>          .
2025-04-30  오후 05:24    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 04:55           412,540 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             414,703 바이트
               2개 디렉터리  144,811,962,368 바이트 남음


In [33]:
import pandas as pd
df = pd.read_csv(r'C:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                encoding='cp949')
df.head(1)

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,...,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,...,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443


In [34]:
new_df = df.melt(id_vars='지역',
           var_name='연도월',
           value_name='평당분양가')
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷)

In [35]:
# air_melted를 와이드 포맷으로
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [36]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                      columns='변수명',
                      values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [37]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수적용시키기
- apply(func, axis) : 축요소별(요소별) 함수 적용(데이터프레임이나 시리즈에서 가능)
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능)
- map(func) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)

## 11.1 apply

In [38]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [39]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [40]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [41]:
get_year('1999-05-01')

1999

In [42]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [43]:
get_month('1999-05-01')

5

In [44]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [45]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.854054
Sepal.Width     3.054730
Petal.Length    3.785034
Petal.Width     1.189726
dtype: float64

In [46]:
X.iloc[0] -avg

Sepal.Length         NaN
Sepal.Width     0.445270
Petal.Length   -2.385034
Petal.Width    -0.989726
dtype: float64

In [47]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,NaN,0.45,-2.39,-0.99
1,-0.95,-0.05,-2.39,-0.99
2,-1.15,0.15,-2.49,-0.99
3,-1.25,0.05,-2.29,-0.99
4,NaN,0.55,-2.39,-0.99
...,...,...,...,...
145,0.85,-0.05,1.41,1.11
146,0.45,-0.55,1.21,0.71
147,0.65,-0.05,1.41,0.81
148,0.35,0.35,1.61,1.11


## 11.2 applymap

In [48]:
X = iris.iloc[:, :-2]

In [49]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,NaN,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,NaN,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [50]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [51]:
# 연습하기(apply, map)
# new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬럼을 추가하세요
# ★ 1차원 시리즈로 처리
new_df.sample()

,지역,연도월,평당분양가
48,경북,2014년2월,6234


In [52]:
new_df['연도'] = new_df['연도월'].apply(lambda str:int(str.split('년')[0]))
new_df['월'] = new_df['연도월'].map(lambda str:int(str.split('년')[1][:-1]))
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      357 non-null    object
 1   연도월     357 non-null    object
 2   평당분양가   357 non-null    int64 
 3   연도      357 non-null    int64 
 4   월       357 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 14.1+ KB


In [53]:
new_df.sample()

,지역,연도월,평당분양가,연도,월
228,경기,2015년1월,10391,2015,1


In [54]:
new_df.drop(['연도','월'], axis=1, inplace=True)
new_df.sample()

,지역,연도월,평당분양가
30,전남,2014년1월,5678


In [54]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [55]:
def split_date(df):
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df

In [56]:
split_date(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [57]:
new_df = new_df.apply(split_date, axis=1)
new_df.sample()

,지역,연도월,평당분양가,연도,월
127,세종,2014년7월,8067,2014,7


In [58]:
new_df.head(2)

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12


In [59]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 reutrn'
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [60]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [61]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


# 12절. 일괄변경하기(결측치, 특정값)

## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [62]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [63]:
# 모든 결측치를 0 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [64]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [65]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [66]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [67]:
# 결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [68]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용
- 데이터 프레임, 시리즈 적용

In [69]:
s = pd.Series([0,1,2,3,np.nan])
s.replace(np.nan, 0)
s.replace([0,1], 99) # 0이나 1을 99로 변경
s.replace([0,1], [100, 101]) # 0은 100으로, 1은 101로 변경

0    100.0
1    101.0
2      2.0
3      3.0
4      NaN
dtype: float64

In [70]:
df = pd.DataFrame({'A': ['bat', 'foo', 'bait'],
                   'B': ['abc', 'bar', 'xyz']})
df.replace(r'ba.+', 'XX', regex=True)

,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건), where(조건)
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치로 출력
- mask(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치로 출력


In [72]:
s = pd.Series(range(10,20))
s

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [73]:
s.where(s>15)


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

In [74]:
s.mask(s>15)

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

## 12.4 dropna
- 결측치가 있는 데이터 누락(행, 열)

In [76]:
df = pd.DataFrame({'name':['홍','김','이', np.nan],
                   'age': [np.nan, 20, 30, np.nan],
                   'born':['서울','부산',np.nan,np.nan]})
df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [77]:
df.dropna(how='all') # 모두 결측치인 행만 삭제 (axis=0기본값)

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [78]:
df.dropna() # 결측치가 하나라도 있는 행 삭제


,name,age,born
1,김,20.0,부산


In [79]:
df.dropna(axis=1) # 결측치가 하나라도 있는 열 삭제


""
0
1
2
3


In [80]:
df.dropna(subset=['name','age'])

,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


# 13절 시리즈
- 1차원 데이터

In [81]:
s = pd.Series([1000, 7000, 5000],
             index=['mango','apple','banana'])
s

mango     1000
apple     7000
banana    5000
dtype: int64

In [82]:
s['mango'], s.mango


(1000, 1000)

In [83]:
s['mango'] = 2800  # 수정
s['orange'] = 4000 # 추가
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [84]:
s['mango':'banana'] # 슬라이싱 [from:to] from부터 to까지(to포함)

mango     2800
apple     7000
banana    5000
dtype: int64

In [85]:
s[ ['mango','banana'] ]

mango     2800
banana    5000
dtype: int64

In [86]:
s.drop('banana')

mango     2800
apple     7000
orange    4000
dtype: int64

In [87]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4000
dtype: int64

In [88]:
s.sort_values() # 오름차순정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [89]:
s.sort_values(ascending=False) # 내림차순

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

In [90]:
s.to_frame().T # s시리즈를 데이터프레임으로 변환하여 전치시킴

,mango,apple,banana,orange
0,2800,7000,5000,4000


# 14절 연습문제 - 실습형

In [91]:
import numpy as np
from seaborn import load_dataset
iris = load_dataset('iris')
type(iris)


pandas.core.frame.DataFrame

In [92]:
# 1. 처음 다섯개 행만 출력
iris.head()
iris.loc[:4]
iris.iloc[:5]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [93]:
# 2. iris 데이터를 독립변수 X와 종속변수 y로. 종속변수는 species. X, y모두 데이터프레임
# iloc이용
X = iris.iloc[:,:-1]
y = iris.iloc[:,-1:] # iris.iloc[:,-1].to_frame()
# loc이용
X = iris.loc[:,'sepal_length':'petal_width']
y=iris.loc[:,'species':] # iris.loc[:,'species'].to_frame()
print(type(X), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [94]:
# 3. iris 데이터에서 처음 50개행을 빼내서 temp변수에 저장하세요
temp = iris.head(50)
temp1 = iris.iloc[:50]
temp2 = iris.loc[:49]
temp3 = iris[:50]
all(temp==temp1), all(temp2==temp3), all(temp==temp2), temp.shape

(True, True, True, (50, 5))

In [95]:
def my_all(*answers):
    if len(answers)>0:
        return all(all(answers[0]==answer) for answer in answers)
    return False

In [96]:
my_all(temp, temp1, temp2, temp3)

True

In [97]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  50 non-null     float64
 1   sepal_width   50 non-null     float64
 2   petal_length  50 non-null     float64
 3   petal_width   50 non-null     float64
 4   species       50 non-null     object 
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [98]:
# 4. 3번에서 선택한 데이터프레임의 요약정보를 출력하세요. 모든 열에 대해 요약정보가 출력
temp.describe(include=['float64','object'])
temp.describe(include='all')


,sepal_length,sepal_width,petal_length,petal_width,species
count,50.00000,50.000000,50.000000,50.000000,50
unique,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.00600,3.428000,1.462000,0.246000,NaN
std,0.35249,0.379064,0.173664,0.105386,NaN
min,4.30000,2.300000,1.000000,0.100000,NaN
25%,4.80000,3.200000,1.400000,0.200000,NaN
50%,5.00000,3.400000,1.500000,0.200000,NaN
75%,5.20000,3.675000,1.575000,0.300000,NaN


In [99]:
import pandas as pd
pd.options.display.max_rows = 5

In [100]:
# 5. versicolor종의 데이터만 iris_versicolor변수에 저장하세요
iris_versicolor = iris.loc[iris.species.str.contains('versicolor')]
iris_versicolor = iris.loc[iris['species']=='versicolor']
iris_versicolor = iris.loc[iris.species=='versicolor']
iris_versicolor = iris[iris.species=='versicolor']
iris_versicolor

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
...,...,...,...,...,...
98,5.1,2.5,3.0,1.1,versicolor
99,5.7,2.8,4.1,1.3,versicolor


In [101]:
# 6. 2번의 X와 y변수를 합해서 iris_df데이터 프레임으로 만드세요(y가 시리즈여도 concat가능)
import pandas as pd
pd.concat([X,y], axis=1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [102]:
# 7. iris 열평균
iris.iloc[:,:-1].mean(axis=0)  # axis=0 기본값 생략 가능
iris.iloc[:,:-1].apply(np.mean, axis=0) # 비추


sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [103]:
# 8. iris 열들 사이의 상관계수
iris.corr(numeric_only=True)
iris.iloc[:,:-1].corr()


,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


In [104]:
# 9. 변수별 평균차이를 출력
iris_mean = iris.iloc[:,:-1].mean() # iris.mean(numeric_only=True)동일
iris_mean


sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [105]:
iris.iloc[:,:-1].apply(lambda x : x-iris_mean, axis=1)

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
...,...,...,...,...
148,0.356667,0.342667,1.642,1.100667
149,0.056667,-0.057333,1.342,0.600667


In [106]:
iris.iloc[:,:-1].subtract(iris_mean)

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
...,...,...,...,...
148,0.356667,0.342667,1.642,1.100667
149,0.056667,-0.057333,1.342,0.600667


In [107]:
# 10. 종별평균
iris.pivot_table(index='species', 
        # values=['sepal_length','sepal_width','petal_length','petal_width'],
        # aggfunc='mean'
        )
iris.groupby(['species']).mean()


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [108]:
# 11 각 요소들과 종별 변수의 평균과의 차이를 각 종별로 3개씩 출력(방법1)
iris_mean_by_species = iris.groupby(iris.species).mean()
iris_mean_by_species


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [109]:
iris.loc[0] - iris_mean_by_species.loc[iris.loc[0].species]

petal_length   -0.062
petal_width    -0.046
sepal_length    0.094
sepal_width     0.072
species           NaN
dtype: object

In [110]:
pd.options.display.max_rows = 9


In [111]:
result = iris.apply(lambda series : series-iris_mean_by_species.loc[series.species], axis=1)
# result.iloc[list(range(3))+list(range(50,53))+list(range(100,103))]
result.iloc[[0,1,2,50,51,52,100,101,102]]
result.groupby(iris.species).head(3)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN


In [112]:
# 11 각 요소들과 종별 변수의 평균과의 차이를 각 종별로 3개씩 출력(방법2)
iris_grouped_sample = iris.groupby('species').head(3)
iris_grouped_sample

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica


In [113]:
iris_mean_by_species

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [114]:
temp = iris_grouped_sample.iloc[0]
temp - iris_mean_by_species.loc[temp.species]


petal_length   -0.062
petal_width    -0.046
sepal_length    0.094
sepal_width     0.072
species           NaN
dtype: object

In [115]:
iris_grouped_sample.apply(lambda temp:temp - iris_mean_by_species.loc[temp.species], 
                         axis=1)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN
